In [2]:
!pip install scapy

  Using cached scapy-2.6.1-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 4.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from scapy.all import sniff
import numpy as np
from sklearn.preprocessing import LabelEncoder

def safe_label_encode(encoder, value):
    
    protocol_mapping = {
    1: "ICMP",
    6: "TCP",
    17: "UDP",
    41: "IPv6",
    50: "ESP",
    51: "AH",
    # Add more from the IANA list if needed
    }   
    
    converted_protocol = protocol_mapping.get(value, "Other")
    
    print(converted_protocol)
    
    return encoder.transform([value])[0]


# Initialize encoders and other necessary tools
protocol_encoder = LabelEncoder()
service_encoder = LabelEncoder()
flag_encoder = LabelEncoder()

# Example mapping for protocol types
protocol_mapping = ['TCP', 'UDP', 'ICMP', 'Other']  # Extend this as needed

protocol_encoder.fit(protocol_mapping)

def extract_features(packet):
    """
    Extract features from a packet.
    """
    
    features = {
        'protocol_type': packet.proto if hasattr(packet, 'proto') else 0,
        'src_bytes': len(packet.payload) if hasattr(packet, 'payload') else 0,
        'dst_bytes': len(packet.payload.payload) if hasattr(packet.payload, 'payload') else 0,
        'flag': packet.sprintf('%TCP.flags%') if 'TCP' in packet else 0,
    }
    
    features['protocol_type'] = safe_label_encode(protocol_encoder, packet.proto if hasattr(packet, 'proto') else 'Other')

    
    return features


In [2]:

import time

batch_size = 16
time_window = 0.4  # 400 ms
batch = []

protocol_mapping = ['TCP', 'UDP', 'ICMP', 'Other']  # Extend this as needed

protocol_encoder.fit(protocol_mapping)

packet_count = 0
last_batch_time = time.time()

def process_packet(packet):
    
    global batch, last_batch_time
    
    #print(packet)
    
    protocol_mapping = {
    1: "ICMP",
    6: "TCP",
    17: "UDP",
    41: "IPv6",
    50: "ESP",
    51: "AH",
    # Add more from the IANA list if needed
    }   
    
    features = {
        'protocol_type': protocol_mapping.get(packet.proto, "Other") if hasattr(packet, 'proto') else 0,
        'src_bytes': len(packet.payload) if hasattr(packet, 'payload') else 0,
        'dst_bytes': len(packet.payload.payload) if hasattr(packet.payload, 'payload') else 0,
        'flag': packet.sprintf('%TCP.flags%') if 'TCP' in packet else 0,
    }
    
    #print(features)

    batch.append(features)
    
    if len(batch) >= batch_size or (time.time() - last_batch_time >= time_window):
        send_batch(batch)
        # Reset the batch and time
        batch = []
        last_batch_time = time.time()
        
def send_batch(batch):
    print(len(batch))




In [3]:
# AUTOENCODER PART

def process_batch(batch):
    """
    Process a batch of packets and perform anomaly detection.
    """
    # Convert to DataFrame
    import pandas as pd
    df = pd.DataFrame(batch)

    # Preprocess features (Label Encoding, Scaling, etc.)
    df['protocol_type'] = protocol_encoder.transform(df['protocol_type'])
    # Repeat for other features

    # Normalize or scale if necessary
    normalized_data = scaler.transform(df.values)  # Assuming a scaler was used in training

    # Run through the autoencoder
    reconstruction = autoencoder.predict(normalized_data)
    reconstruction_error = np.mean(np.square(normalized_data - reconstruction), axis=1)

    # Detect anomalies
    anomalies = reconstruction_error > threshold
    handle_anomalies(df, anomalies)

def handle_anomalies(data, anomalies):
    """
    Log or respond to anomalies.
    """
    for idx, is_anomaly in enumerate(anomalies):
        if is_anomaly:
            print(f"Anomaly detected: {data.iloc[idx]}")
            # Add additional actions like logging or alerting

In [4]:
#sniff(filter="tcp or udp", prn=process_packet, store=0)
#batch_size = max(32, len(batch) // 10)
sniff(filter="ip", prn=lambda x: process_packet(x), count=0)

#sniff(filter="ip", prn=lambda x: print(f"Captured: {x.summary()}"))


1
16
16
16
16
16
6
5
13
8
12
8
16
16
16
16
16
16
16
16
16
16
16
11
16
16
16


<Sniffed: TCP:361 UDP:21 ICMP:0 Other:0>

In [ ]:
# if you have sniffing blocking or any other Network Traffic Limitations

import threading

def start_sniffing():
    sniff(filter="ip", prn=process_packet)


while True: threading.Thread(target=start_sniffing).start()


[{'protocol_type': 'TCP', 'src_bytes': 560, 'dst_bytes': 540, 'flag': 'PA'}][{'protocol_type': 'TCP', 'src_bytes': 560, 'dst_bytes': 540, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 560, 'dst_bytes': 540, 'flag': 'PA'}]
[{'protocol_type': 'TCP', 'src_bytes': 83, 'dst_bytes': 63, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 91, 'dst_bytes': 71, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 151, 'dst_bytes': 131, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 91, 'dst_bytes': 71, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 489, 'dst_bytes': 469, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 560, 'dst_bytes': 540, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 83, 'dst_bytes': 63, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 91, 'dst_bytes': 71, 'flag': 'PA'}, {'protocol_type': 'TCP', '

Exception in thread Exception in thread Thread-13124 (start_sniffing):
Traceback (most recent call last):
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-13122 (start_sniffing):
Traceback (most recent call last):
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Thread-13130 (start_sniffing):
Traceback (most recent call last):
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-13123 (start_sniffing):
Traceback (most recent call last):
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-13125 (start_sniffing):
Traceback (most recent call last):
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/andriimatsevytyi/anaconda

[{'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'UDP', 'src_bytes': 65, 'dst_bytes': 45, 'flag': 0}][{'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes

        self._target(*self._args, **self._kwargs)
  File "/var/folders/s7/vf054wld4d31cr3ltwm243cr0000gn/T/ipykernel_36630/2452383236.py", line 6, in start_sniffing
    self._target(*self._args, **self._kwargs)
  File "/var/folders/s7/vf054wld4d31cr3ltwm243cr0000gn/T/ipykernel_36630/2452383236.py", line 6, in start_sniffing
sniff_sockets[_RL2(iface)(type=ETH_P_ALL, iface=iface,
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/site-packages/scapy/arch/bpf/supersocket.py", line 365, in __init__
Exception in thread Thread-13319 (start_sniffing):
Traceback (most recent call last):
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.bpf_fd, self.dev_bpf = get_dev_bpf()
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/site-packages/scapy/arch/bpf/core.py", line 68, in get_dev_bpf
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/site-packages/scapy/sendrecv.py", line 1424, in sniff
Exception in thread Thread-1

[{'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 1280, 'dst_bytes': 1260, 'flag': 'A'}][{'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 1280, 'dst_bytes': 1260, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 1280, 'dst_bytes': 1260, 'flag': 'A'}]
[{'protocol_type': 'TCP', 'src_bytes': 921, 'dst_bytes': 901, 'flag': 'PA'}, {'protocol_type': 'TCP', 'src_bytes': 52, 'dst_bytes': 32, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 1280, 'dst_bytes': 1260, 'flag': 'A'}, {'protocol_type': 'TCP', 'src_bytes': 921, 'dst_bytes': 901, 'flag': 'PA'}, {'protocol_type': 'TCP', 's

        self.bpf_fd, self.dev_bpf = get_dev_bpf()
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/site-packages/scapy/arch/bpf/core.py", line 68, in get_dev_bpf
    sniff_sockets[_RL2(iface)(type=ETH_P_ALL, iface=iface,
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/site-packages/scapy/arch/bpf/supersocket.py", line 365, in __init__
    self.run()
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/threading.py", line 953, in run
    super(L2bpfListenSocket, self).__init__(*args, **kwargs)
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/site-packages/scapy/arch/bpf/supersocket.py", line 129, in __init__
    self.bpf_fd, self.dev_bpf = get_dev_bpf()
  File "/Users/andriimatsevytyi/anaconda3/lib/python3.10/site-packages/scapy/arch/bpf/core.py", line 68, in get_dev_bpf
    raise Scapy_Exception("No /dev/bpf handle is available !")
scapy.error.Scapy_Exception: No /dev/bpf handle is available !
    self._target(*self._args, **self._kwargs)
  File "/var/folde